In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
###############################################################################
#                                                                             #
# Copyright (C) 2010 Edward d'Auvergne                                        #
#                                                                             #
# This file is part of the program relax.                                     #
#                                                                             #
# relax is free software; you can redistribute it and/or modify               #
# it under the terms of the GNU General Public License as published by        #
# the Free Software Foundation; either version 2 of the License, or           #
# (at your option) any later version.                                         #
#                                                                             #
# relax is distributed in the hope that it will be useful;                    #
# but WITHOUT ANY WARRANTY; without even the implied warranty of              #
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the               #
# GNU General Public License for more details.                                #
#                                                                             #
# You should have received a copy of the GNU General Public License           #
# along with relax; if not, write to the Free Software                        #
# Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA  02111-1307  USA   #
#                                                                             #
###############################################################################

# Module docstring.
"""Module for transforming between different coordinate systems."""

# Python module imports.
from math import acos, atan2, cos, sin
from numpy import array, float64, zeros
from numpy.linalg import norm


def cartesian_to_spherical(vector):
    """Convert the Cartesian vector [x, y, z] to spherical coordinates [r, theta, phi].

    The parameter r is the radial distance, theta is the polar angle, and phi is the azimuth.


    @param vector:  The Cartesian vector [x, y, z].
    @type vector:   numpy rank-1, 3D array
    @return:        The spherical coordinate vector [r, theta, phi].
    @rtype:         numpy rank-1, 3D array
    """

    # The radial distance.
    r = norm(vector)

    # Unit vector.
    unit = vector / r

    # The polar angle.
    theta = acos(unit[2])

    # The azimuth.
    phi = atan2(unit[1], unit[0])

    # Return the spherical coordinate vector.
    return array([r, theta, phi], float64)


def spherical_to_cartesian(spherical_vect, cart_vect):
    """Convert the spherical coordinate vector [r, theta, phi] to the Cartesian vector [x, y, z].

    The parameter r is the radial distance, theta is the polar angle, and phi is the azimuth.


    @param spherical_vect:  The spherical coordinate vector [r, theta, phi].
    @type spherical_vect:   3D array or list
    @param cart_vect:       The Cartesian vector [x, y, z].
    @type cart_vect:        3D array or list
    """

    # Trig alias.
    sin_theta = sin(spherical_vect[1])

    # The vector.
    cart_vect[0] = spherical_vect[0] * cos(spherical_vect[2]) * sin_theta
    cart_vect[1] = spherical_vect[0] * sin(spherical_vect[2]) * sin_theta
    cart_vect[2] = spherical_vect[0] * cos(spherical_vect[1])

# Define an equator-centerd domain

In [ ]:
lat_min = -20.0
lat_max = 20.0
lon_min = 109.0
lon_max = 159.0
nlat = 41
nlon = 51
deg2rad = np.pi/180.0
rad2deg = 1.0/deg2rad

In [ ]:
lati = np.linspace(lat_min, lat_max, nlat)
loni = np.linspace(lon_min, lon_max, nlon)

In [ ]:
domain = np.zeros((nlon*nlat,3))
for j in range(nlat):
    for i in range(nlon):
        n = i + nlon*j
        spherical_vect = [1.0, deg2rad*(90.0-lati[j]), deg2rad*loni[i]]
        spherical_to_cartesian(spherical_vect, domain[n,:])

From http://stackoverflow.com/a/25709323,

    let a be the unit vector along axis, i.e. a = axis/norm(axis)
    and A = I × a be the skew-symmetric matrix associated to a, i.e. the cross product of the identity matrix with a
    then M = exp(θ A) is the rotation matrix.
    
Example:
```Python
from numpy import cross, eye, dot
from scipy.linalg import expm3, norm

def M(axis, theta):
    return expm3(cross(eye(3), axis/norm(axis)*theta))

v, axis, theta = [3,5,0], [4,4,1], 1.2
M0 = M(axis, theta)

print(dot(M0,v))
# [ 2.74911638  4.77180932  1.91629719]
```

In [ ]:
from numpy import cross, eye, dot
from scipy.linalg import expm, norm

def M(axis, theta):
    return expm(theta*cross(eye(3),axis/norm(axis)))

In [ ]:
# The rotation axis is pointing to longitude of (136-90) deg and latitude of 0 deg.
# The dessired rotation is +36 deg around this axis.

# 1. Compute the x, y, z coordinates of the rotation axis.
axis_latlon = np.array([1.0, deg2rad*90.0, deg2rad*(0.5*(lon_max+lon_min)-90.0)])
axis = np.array([0.0, 0.0, 0.0])
spherical_to_cartesian(axis_latlon, axis)

# 2. Define the rotation matrix
theta = deg2rad*36.0
M0 = M(axis, theta)

# 3. Rotate the domain
domain_rot = np.zeros((nlon*nlat,3))
for j in range(nlat):
    for i in range(nlon):
        n = i + nlon*j
        domain_rot[n,:] = dot(M0,domain[n,:])

# 4. Convert the x, y, z to lon-lat
domain_rot_latlon = np.zeros((nlon*nlat,2))
for j in range(nlat):
    for i in range(nlon):
        n = i + nlon*j
        r, theta, phi = cartesian_to_spherical(domain_rot[n,:])
        # latitude
        domain_rot_latlon[n,:] = [90.0-rad2deg*theta, rad2deg*phi]

In [ ]:
with open('domain.latlon','w') as fo1:
    with open('domain_rot.latlon','w') as fo2:
        for j in range(nlat):
            for i in range(nlon):
                n = i + nlon*j
                print("{0:g} {1:g}".format(loni[i],lati[j]), file=fo1)
                print("{0:g} {1:g}".format(domain_rot_latlon[n,1], domain_rot_latlon[n,0]), file=fo2)


## Epicenter rotation

In [ ]:
# 0. Epicenter of the 2011 Tohoku earthquake
epi_lon = 142.86
epi_lat = 38.103

# 1. Compute the x, y, z coordinates of the epicenter
epi_latlon = np.array([1.0, deg2rad*(90.0-epi_lat), deg2rad*epi_lon])
epi_xyz    = np.array([0.0, 0.0, 0.0])
spherical_to_cartesian(epi_latlon, epi_xyz)

# 2. Define the rotation matrix
theta = deg2rad*-36.0
M1 = M(axis, theta)

# 3. Rotate the epicenter
epi_rot_xyz = dot(M1,epi_xyz)

# 4. Convert the x, y, z to lon-lat
r, theta, phi = cartesian_to_spherical(epi_rot_xyz)
epi_rot_latlon = [90.0-rad2deg*theta, rad2deg*phi]
print(epi_rot_latlon)

## Coastline rotation

In [ ]:
# 0. Load coastline lat lon
with open('./Coastlines/coastlines.latlon','r') as fi:
    lines = fi.readlines()

# Open an output file.
with open('./Coastlines/coastlines_rot.latlon','w') as fo:
    for line in lines:
        # 0. skip segment headers
        if line.startswith('>'):
            print(line.rstrip(), file=fo)
            continue
        slatlon = line.split()

        # 1. Compute the x, y, z coordinates of the coastline segments
        lon, lat = float(slatlon[0]), float(slatlon[1])
        latlon = np.array([1.0, deg2rad*(90.0-lat), deg2rad*lon])
        xyz    = np.array([0.0, 0.0, 0.0])
        spherical_to_cartesian(latlon, xyz)

        # 2. Define the rotation matrix. Reuse M1. Need not make it again.

        # 3. Rotate the epicenter
        rot_xyz = dot(M1,xyz)
        #print xyz, rot_xyz

        # 4. Convert the x, y, z to lon-lat
        r, theta, phi = cartesian_to_spherical(rot_xyz)
        rot_latlon = [90.0-rad2deg*theta, rad2deg*phi]
        print("{0:g} {1:g}".format(rot_latlon[1], rot_latlon[0]), file=fo)
